In [18]:
#TechVidvan
# load all required libraries for Dog's Breed Identification Project
import cv2
import numpy as np 
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

In [21]:
#read the csv file
df_labels = pd.read_csv(r"C:\Users\Artsy\Desktop\Dog_Breed_Categorization\labels.csv")
#store training and testing images folder location
train_file = 'train/'
test_file = 'test/'

In [22]:
#check the total number of unique breed in our dataset file
print("Total number of unique Dog Breeds :",len(df_labels.breed.unique()))

Total number of unique Dog Breeds : 8


In [23]:
#specify number
num_breeds = 8
im_size = 224
batch_size = 64
encoder = LabelEncoder()


In [24]:
#get only 60 unique breeds record 
breed_dict = list(df_labels['breed'].value_counts().keys()) 
new_list = sorted(breed_dict,reverse=True)[:num_breeds*2+1:2]
#change the dataset to have only those 60 unique breed records
df_labels = df_labels.query('breed in @new_list')

In [25]:
#create new column which will contain image name with the image extension
df_labels['img_file'] = df_labels['id'].apply(lambda x: x + ".jpg")

In [26]:
df_labels

,Unnamed: 0,id,breed,img_file
4,70,01b36cb1b80ab8c3a7d2b7128ad21bdc,samoyed,01b36cb1b80ab8c3a7d2b7128ad21bdc.jpg
5,84,0206c12e8984e3c8a166cc272de25d6f,chihuahua,0206c12e8984e3c8a166cc272de25d6f.jpg
6,92,023e4e28415506e0deddcbd8f8bdab29,pug,023e4e28415506e0deddcbd8f8bdab29.jpg
7,97,02508e76981e1ba059d785704b4c480c,pug,02508e76981e1ba059d785704b4c480c.jpg
13,147,037ac6cbd9c96d70e68917aa059b6aab,old_english_sheepdog,037ac6cbd9c96d70e68917aa059b6aab.jpg
...,...,...,...,...
729,10035,fba73e53151cb751f9c22e63de669923,pug,fba73e53151cb751f9c22e63de669923.jpg
733,10137,fdcbedc65f600f81df181e9a46858cbb,chihuahua,fdcbedc65f600f81df181e9a46858cbb.jpg
734,10150,fe0beca881efd723510e8e859306a3a6,pug,fe0beca881efd723510e8e859306a3a6.jpg
736,10170,feb16cf86c9dac6d476e3c372ba5c279,pug,feb16cf86c9dac6d476e3c372ba5c279.jpg


In [27]:
#create a numpy array of the shape
#(number of dataset records, image size , image size, 3 for rgb channel ayer)
#this will be input for model
train_x = np.zeros((len(df_labels), im_size, im_size, 3), dtype='float32')
 
#iterate over img_file column of our dataset
for i, img_id in enumerate(df_labels['img_file']):
  #read the image file and convert into numeric format
  #resize all images to one dimension i.e. 224x224
  #we will get array with the shape of
  # (224,224,3) where 3 is the RGB channels layers
  img = cv2.resize(cv2.imread(train_file+img_id,cv2.IMREAD_COLOR),((im_size,im_size)))
  #scale array into the range of -1 to 1.
  #preprocess the array and expand its dimension on the axis 0 
  img_array = preprocess_input(np.expand_dims(np.array(img[...,::-1].astype(np.float32)).copy(), axis=0))
  #update the train_x variable with new element
  train_x[i] = img_array

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
